In [1]:
# Install Gradio for the interactive dashboard
!pip install gradio

In [18]:
from google.colab import drive
drive.mount('/content/drive')

# This is the path to the folder you created
base_dir = '/content/drive/MyDrive/cow_lumpy_dataset'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import tensorflow as tf
import os

# --- Configuration ---
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 1337
# --- End Configuration ---

# Load the dataset and split it into training (80%) and (validation + test) (20%)
print("Loading training data (80%)...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)

print("Loading validation and test data (20%)...")
val_test_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)

# Now, split the val_test_ds into validation (10%) and test (10%)
val_batches = tf.data.experimental.cardinality(val_test_ds)
test_ds = val_test_ds.take(val_batches // 2)
val_ds = val_test_ds.skip(val_batches // 2)

print(f"Training batches: {tf.data.experimental.cardinality(train_ds)}")
print(f"Validation batches: {tf.data.experimental.cardinality(val_ds)}")
print(f"Test batches: {tf.data.experimental.cardinality(test_ds)}")

# Get the class names
class_names = train_ds.class_names
print(f"\nFound classes: {class_names}")

# Configure all three datasets for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Loading training data (80%)...
Found 2513 files belonging to 2 classes.
Using 2011 files for training.
Loading validation and test data (20%)...
Found 2513 files belonging to 2 classes.
Using 502 files for validation.
Training batches: 63
Validation batches: 8
Test batches: 8

Found classes: ['hcaugmented-20251114T125321Z-1-001', 'lcaugmented-20251114T125417Z-1-001']


In [22]:
# Data augmentation layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
], name="data_augmentation")

# Load the pre-trained MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMAGE_SIZE + (3,),
    include_top=False,  # Do not include its final classification layer
    weights='imagenet'
)

# --- FREEZE THE BASE MODEL ---
# This is the key part of transfer learning (non-fine-tuning)
base_model.trainable = False

# Build our new model on top
inputs = tf.keras.Input(shape=IMAGE_SIZE + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x) # Preprocess for MobileNetV2
x = base_model(x, training=False) # Run base model in inference mode
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x) # Binary output

model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("--- Model Summary (Base Frozen) ---")
model.summary()

--- Model Summary (Base Frozen) ---


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_2 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract_2 (Subtract)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [23]:
EPOCHS_HEAD = 10

print("\n--- Starting Head Training (Base Frozen) ---")
history_head = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_HEAD
)
print("--- Head Training Complete ---")


--- Starting Head Training (Base Frozen) ---
Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 414s 6s/step - accuracy: 0.6560 - loss: 0.6261 - val_accuracy: 0.8455 - val_loss: 0.4280
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.8159 - loss: 0.4216 - val_accuracy: 0.8740 - val_loss: 0.3774
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - accuracy: 0.8336 - loss: 0.3790 - val_accuracy: 0.8780 - val_loss: 0.3569
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - accuracy: 0.8478 - loss: 0.3507 - val_accuracy: 0.8780 - val_loss: 0.3440
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.8520 - loss: 0.3430 - val_accuracy: 0.8780 - val_loss: 0.3366
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.8659 - loss: 0.3215 - val_accuracy: 0.8780 - val_loss: 0.3337
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - accuracy: 0.8755 - loss: 0.3003 - val_accuracy: 0.8902 - val_loss: 0.3269
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.

In [24]:
print("\n--- Starting Fine-Tuning (Unfreezing Top Layers) ---")

# --- 1. Unfreeze the base model ---
base_model.trainable = True

# We'll unfreeze the top 50 layers. You can experiment with this number.
# The bottom layers (which detect simple edges/colors) should stay frozen.
fine_tune_at_layer = 100  # MobileNetV2 has 154 layers

for layer in base_model.layers[:fine_tune_at_layer]:
    layer.trainable = False

# --- 2. Re-compile with a low learning rate ---
# This is crucial. If the LR is too high, it will destroy the pre-trained weights.
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # 0.00001
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("--- Model Summary (Fine-Tuning) ---")
model.summary() # Note the massive increase in "Trainable params"

# --- 3. Add Callbacks ---
# We use EarlyStopping to stop when the model is at its best
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', # Stop when validation loss stops improving
        patience=3,          # Wait 3 epochs for improvement
        restore_best_weights=True # Automatically restore the model from the best epoch
    )
]

# --- 4. Continue training (fine-tuning) ---
EPOCHS_FINE_TUNE = 15
total_epochs = EPOCHS_HEAD + EPOCHS_FINE_TUNE

history_fine_tune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=total_epochs,
    initial_epoch=history_head.epoch[-1], # Start from where we left off
    callbacks=callbacks # Use the EarlyStopping callback
)

print("--- Fine-Tuning Complete ---")


--- Starting Fine-Tuning (Unfreezing Top Layers) ---
--- Model Summary (Fine-Tuning) ---


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_2 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract_2 (Subtract)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,862,721 (7.11 MB)

 Non-trainable params: 396,544 (1.51 MB)

Epoch 10/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 21s 131ms/step - accuracy: 0.8225 - loss: 0.3935 - val_accuracy: 0.8821 - val_loss: 0.2967
Epoch 11/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 96ms/step - accuracy: 0.8618 - loss: 0.3256 - val_accuracy: 0.8902 - val_loss: 0.2946
Epoch 12/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 95ms/step - accuracy: 0.8709 - loss: 0.3033 - val_accuracy: 0.8862 - val_loss: 0.2818
Epoch 13/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 97ms/step - accuracy: 0.8840 - loss: 0.2830 - val_accuracy: 0.8902 - val_loss: 0.2706
Epoch 14/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 95ms/step - accuracy: 0.9003 - loss: 0.2547 - val_accuracy: 0.8984 - val_loss: 0.2589
Epoch 15/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 98ms/step - accuracy: 0.9047 - loss: 0.2366 - val_accuracy: 0.9024 - val_loss: 0.2482
Epoch 16/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 96ms/step - accuracy: 0.9049 - loss: 0.2283 - val_accuracy: 0.9065 - val_loss: 0.2390
Epoch 17/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 97ms/step - accuracy: 0.9210 - loss: 0.2125 - val_accuracy: 

In [25]:
print("\n--- Evaluating Final Model on Unseen Test Data ---")
loss, accuracy = model.evaluate(test_ds)
print(f"Final Test Accuracy: {accuracy * 100:.2f}%")
print(f"Final Test Loss: {loss:.4f}")


--- Evaluating Final Model on Unseen Test Data ---
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - accuracy: 0.9397 - loss: 0.1754
Final Test Accuracy: 94.14%
Final Test Loss: 0.1531


In [27]:
import os
import shutil
from google.colab import files
import tensorflow as tf

print("--- Preparing downloadable datasets ---")

# --- Function to save a dataset to a directory ---
def save_dataset_to_dir(dataset, dir_name, class_names):

    # Create the root directory (e.g., 'training_dataset')
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name) # Clear old one if it exists
    os.makedirs(dir_name)

    # Create the sub-directories for each class
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

    print(f"Saving images to '{dir_name}'...")

    # Iterate through the dataset
    # We use .unbatch() to process images one by one
    image_count = 0
    for images, labels in dataset.unbatch():
        # The label is an integer (0 or 1)
        class_name = class_names[labels]

        # Create a unique filename
        filename = f"image_{image_count}.png"
        filepath = os.path.join(dir_name, class_name, filename)

        # Save the image
        tf.keras.utils.save_img(filepath, images)
        image_count += 1

    print(f"Successfully saved {image_count} images to '{dir_name}'.")

# --- 1. Save the training and test datasets ---
# (Make sure 'train_ds', 'test_ds', and 'class_names' exist from Step 3)
try:
    save_dataset_to_dir(train_ds, "training_dataset", class_names)
    save_dataset_to_dir(test_ds, "test_dataset", class_names)

    # --- 2. Zip the new folders ---
    print("\nZipping folders...")
    shutil.make_archive("training_dataset", 'zip', "training_dataset")
    print("Created training_dataset.zip")

    shutil.make_archive("test_dataset", 'zip', "test_dataset")
    print("Created test_dataset.zip")

    # --- 3. Download the zip files ---
    print("\nTriggering downloads...")
    files.download("training_dataset.zip")
    files.download("test_dataset.zip")

except NameError:
    print("\n--- ERROR ---")
    print("Please run Step 3 (Prepare Data Pipeline) from the previous response")
    print("before running this cell, so that 'train_ds' and 'test_ds' are defined.")

--- Preparing downloadable datasets ---
Saving images to 'training_dataset'...
Successfully saved 2011 images to 'training_dataset'.
Saving images to 'test_dataset'...
Successfully saved 256 images to 'test_dataset'.

Zipping folders...
Created training_dataset.zip
Created test_dataset.zip

Triggering downloads...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import gradio as gr
import numpy as np
from PIL import Image

# Use the class_names we found in Step 3
# Create a mapping for clear labels
# Keras labels alphabetically: 'hcaugmented' = 0, 'lcaugmented' = 1
# So, a prediction < 0.5 is 'hcaugmented' (Healthy)
# and a prediction > 0.5 is 'lcaugmented' (Lumpy Disease)

display_labels = {
    class_names[0]: "Healthy",      # Assuming class_names[0] is 'hcaugmented'
    class_names[1]: "Lumpy Disease" # Assuming class_names[1] is 'lcaugmented'
}


def predict_image(img):
    """Prediction function for the Gradio interface."""
    img_array = np.array(img)
    img_resized = tf.image.resize(img_array, IMAGE_SIZE)
    img_batch = tf.expand_dims(img_resized, 0) # Create a batch

    # Get prediction (a value between 0 and 1)
    prediction = model.predict(img_batch)[0][0]

    if prediction < 0.5:
        # Class 0
        label_name = display_labels[class_names[0]]
        confidence = 1 - prediction
    else:
        # Class 1
        label_name = display_labels[class_names[1]]
        confidence = prediction

    # Flip the logic for the "other" class to show both
    other_label_name = "Lumpy Disease" if label_name == "Healthy" else "Healthy"

    return {label_name: float(confidence), other_label_name: float(1 - confidence)}


# Create the Gradio interface
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload Cow Image"),
    outputs=gr.Label(num_top_classes=2, label="Prediction"),
    title="🐄 Cow Lumpy Disease Predictor (Fine-Tuned) 🐄",
    description="Upload an image of a cow to predict if it is Healthy or has Lumpy Skin Disease. This model uses a fine-tuned MobileNetV2.",
    allow_flagging="never"
)

# Launch the dashboard!
iface.launch(debug=True)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://74b60538963ef59b21.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
